# Watson Attempt 2

### Imports

In [1]:
import os
import shutil
import json

from whoosh.fields import Schema, TEXT, ID
from whoosh.index import create_in, open_dir, exists_in
from whoosh.qparser import QueryParser, OrGroup
from whoosh.scoring import BM25F
from whoosh.analysis import RegexTokenizer, LowercaseFilter, NgramFilter

In [2]:
from env import env
from shared import wiki_df, questions_df, LemmatizeFilter, filter_query # importing from shared takes a few seconds

In [3]:
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = input("Enter your OpenAI API Key: ")

In [4]:
def query_ChatGPT(query):
    import requests

    model = "gpt-3.5-turbo-0301"

    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + api_key,
    }

    json_data = {
            "model": model,
            "temperature": 0,
            "messages": [
                {
                    "role": "user",
                    "content": query
                }
            ]
        }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=json_data).json()
    assert "choices" in response, response
    assert len(response["choices"]) > 0, response
    assert "message" in response["choices"][0], response
    assert "content" in response["choices"][0]["message"], response
    
    return response["choices"][0]["message"]["content"]

In [5]:
with open("ChatGPT_template/t3.txt", "r") as file:
    template1 = file.read()
def pass_query_through_ChatGPT(query):
    gptq = template1 + query
    try:
        return query_ChatGPT(gptq)
    except:
        return query

In [6]:
with open("ChatGPT_template/t2.txt", "r") as file:
    template2 = file.read()
def boost_important_terms(query):
    if len(query.split()) <= 3:
        return query
    
    gptq = template2 + query

    try:
        terms = json.loads(query_ChatGPT(gptq))
    except:
        return query
    
    if len(terms) < 3:
        return query
    
    try:
        term1 = terms["term1"]
        term2 = terms["term2"]
        term3 = terms["term3"]
    except:
        return query
    
    # boost the importance of the 3 least frequent terms TODO tune the boost level
    query = query.replace(term1, term1 + "^1.7")
    query = query.replace(term2, term2 + "^1.5")
    query = query.replace(term3, term3 + "^1.3")
    return query

In [7]:
def query_pipeline(query):
    # query = pass_query_through_ChatGPT(query)
    query = filter_query(query)
    # query = boost_important_terms(query)
    return query

In [8]:
def category_pipeine(category):
    new_cat = ""
    for c in category.split():
        new_cat += c + "^0.5 "
    return new_cat.strip()

In [9]:
with open("ChatGPT_template/t4.txt", "r") as file:
    template4 = file.read()
def rerank_results(question, results):
    k = 10
    data = { "question": question, "guesses": [] }
    for i in range(k):
        data["guesses"].append(results[i][0])
    gptq = template4 + json.dumps(data)
    try:
        reranked_guesses = json.loads(query_ChatGPT(gptq))["reranked_guesses"]
        reranked_results = []
        for i in range(k):
            reranked_results.append((reranked_guesses[i], i + 1))
        reranked_results += results[k+1:]
        return reranked_results
    except:
        return results

### Define the Watson class

In [10]:
class Watson:
    def __init__(self):
        self.Q = len(questions_df.index)
        self._analyzer = self._build_analyzer()
        self._index = self._build_index()
        self._parser = self._build_parser()

    def _build_analyzer(self):
        return RegexTokenizer() | LowercaseFilter() | LemmatizeFilter()
    
    def _build_index(self):
        if exists_in(env.index_path):
            ix = open_dir(env.index_path)
        else:
            if os.path.exists(env.index_path):
                shutil.rmtree(env.index_path)
            os.mkdir(env.index_path)
            schema = Schema(title=ID(stored=True),  
                    titles=TEXT(analyzer=self._analyzer), 
                    categories=TEXT(analyzer=self._analyzer), 
                    content=TEXT(analyzer=self._analyzer))
            ix = create_in(env.index_path, schema)
            with ix.writer() as writer:
                for _, row in wiki_df.iterrows():
                    writer.add_document(title=row.title, content=row.text)
        return ix
    
    def _build_parser(self):
        og = OrGroup.factory(0.9)
        return QueryParser("content", schema=self._index.schema, group=og)
    
    def search(self, category, question, scorer=BM25F):
        try:
            category = category_pipeine(category)
            question = query_pipeline(question)
            query = self._parser.parse(category + " " + question)
        except TypeError as e:
            print(query_pipeline(question))
            raise e
        with self._index.searcher(weighting=scorer()) as searcher:
            results = searcher.search(query, limit=None)
            if results.scored_length() == 0:
                return None
            return rerank_results(question, [(r["title"], r.rank+1) for r in results])

    def test(self, scorer=BM25F, eval="mrr"):
        if eval == "mrr":
            mrr = 0.0
            for _, row in questions_df.iterrows():
                results = self.search(row.category, row.question, scorer)
                rank = Watson.get_rank(results, row.answer)
                if rank > 0:
                    mrr += 1 / rank
            mrr /= self.Q
            return mrr
        elif eval == "p@1":
            correct = 0
            for _, row in questions_df.iterrows():
                results = self.search(row.category, row.question, scorer)
                if Watson.is_correct(results, row.answer):
                    correct += 1
            return correct / self.Q
        elif eval == "both":
            mrr = 0.0
            correct = 0
            for _, row in questions_df.iterrows():
                results = self.search(row.category, row.question, scorer)
                rank = Watson.get_rank(results, row.answer)
                if rank > 0:
                    mrr += 1 / rank
                if Watson.is_correct(results, row.answer):
                    correct += 1
            return mrr / self.Q, correct / self.Q
        else:
            raise Exception(f"unrecognized evaluation type: {eval}")
        
    @staticmethod
    def get_rank(results, answer):
        for answer_variant in answer.split("|"):
            for (doc_title, rank) in results:
                if doc_title.lower() == answer_variant.lower():
                    return rank
        return 0
    
    @staticmethod
    def is_correct(results, answer):
        guess, _ = results[0]
        for answer_variant in answer.split("|"):
            if answer_variant.lower() == guess.lower():
                return True
        return False
    
    @staticmethod
    def get_guess(results):
        guess, _ = results[0]
        return guess
    

### Instantiate Watson

In [11]:
watson = Watson()

### Test Watson

In [12]:
mrr_score, pa1_score = watson.test(eval="both")
mrr_score, pa1_score

I am going to give you a JSON string that looks like this:
{
    "question": "<question>"
    "quesses": [
        "<quess1>",
        "<guess2>",
        .
        .
        .
    ]
}
Where <question> is a fact about an unknown object, and guesses is a list of quesses for what the object could be, ranked.

I want you to rerank the list of guesses and return a better ranking of guesses for the question, also in JSON format, like this:
{
    "reranked_guesses": [
        "<quess1>",
        "<guess2>",
        .
        .
        .
    ]
}

The rest of the input after this sentence is the JSON string.
{"question": "dominant paper our nation's capital, it's among top 10 U.S. papers circulation", "guesses": ["Media Of The United States", "The New York Sun", "Star Tribune", "The Washington Post", "Sun-Sentinel", "Los Angeles Times", "People'S Daily", "The Wall Street Journal", "William Randolph Hearst", "Irish Independent"]}
here
I am going to give you a JSON string that looks like this:
{

(0.5611990159725353, 0.53)

### TODO
- Finish Watson attempt 2
- Update scores in report
- Create presentation